[1] "sdfs"


In [77]:
cases %>% 
  filter(time > t_start & time < (t_end + 0.01))

date,dep,cases,time
2015-06-06,Artibonite,26,2015.426
2015-06-13,Artibonite,58,2015.446
2015-06-20,Artibonite,85,2015.465
2015-06-27,Artibonite,65,2015.484
2015-07-04,Artibonite,63,2015.503
2015-07-11,Artibonite,44,2015.522
2015-07-18,Artibonite,37,2015.541
2015-07-25,Artibonite,52,2015.561
2015-08-01,Artibonite,99,2015.580
2015-08-08,Artibonite,58,2015.599


In [74]:
cases

date,dep,cases
2010-10-23,Artibonite,6234
2010-10-30,Artibonite,7603
2010-11-6,Artibonite,4835
2010-11-13,Artibonite,8589
2010-11-20,Artibonite,7097
2010-11-27,Artibonite,5025
2010-12-4,Artibonite,3763
2010-12-11,Artibonite,2451
2010-12-18,Artibonite,2429
2010-12-25,Artibonite,2583


In [41]:
typeof(select(rain, departement)/max_rain)

[1] "list"

In [45]:
rain

date,Artibonite,time
2000-03-01,0.000,2000.163
2000-03-02,0.000,2000.166
2000-03-03,0.000,2000.168
2000-03-04,0.000,2000.171
2000-03-05,23.340,2000.174
2000-03-06,3.876,2000.177
2000-03-07,1.990,2000.179
2000-03-08,0.000,2000.182
2000-03-09,0.564,2000.185
2000-03-10,1.056,2000.188


In [3]:
rain <- read_csv("haiti-data/fromAzman/rainfall.csv") %>%  select('date', 'Artibonite') %>%
mutate(date = as.Date(date, format = "%Y-%m-%d"),
   time = dateToYears(date)) 

Parsed with column specification:
cols(
  date = col_date(format = ""),
  Artibonite = col_double(),
  Centre = col_double(),
  Grande_Anse = col_double(),
  Nippes = col_double(),
  Nord = col_double(),
  `Nord-Est` = col_double(),
  `Nord-Ouest` = col_double(),
  Ouest = col_double(),
  Sud = col_double(),
  `Sud-Est` = col_double()
)


In [4]:
max_rain <- rain %>%
#filter(year(date)==2015 & month(date) < 10) %>% 
select(departement) %>% 
max()
rain %<>% mutate(rain_std = select(rain, departement)/max_rain)

ERROR: Error in mutate_impl(.data, dots): Column `rain_std` is of unsupported class data.frame


In [2]:
select(rain, departement)/max_rain

ERROR: Error: Unknown column `Artibonite` 


In [25]:
rain

date,Artibonite,time
2000-03-01,0.000,2000.163
2000-03-02,0.000,2000.166
2000-03-03,0.000,2000.168
2000-03-04,0.000,2000.171
2000-03-05,23.340,2000.174
2000-03-06,3.876,2000.177
2000-03-07,1.990,2000.179
2000-03-08,0.000,2000.182
2000-03-09,0.564,2000.185
2000-03-10,1.056,2000.188
